## Basic ReAct Agent using Langchain

In [1]:
# Import Block
import openai
import re
import httpx
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
from openai import OpenAI 
# Initiate Open AI Client
client = OpenAI()

## Explanation of Agent Implementation
- The `__init__` function is the constructor, which always has sytem prompt built in it. 
- It also saves messages in the `messages` variable. Start with the system message
- The `__call__` function makes the Agent class callable, and executes the ai client call and appends results to messages
- The following is the basic building block of agent call, no ReAct implementation yet

In [2]:
# Create Agent Class with constructor and make it callable
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self,message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        chat_completion = client.chat.completions.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=self.messages,
        )
        return chat_completion.choices[0].message.content
    

### ReAct Agent Implementation
- ReAct is a concent where the ai client is called in loops through the workflow : 
    1. Thought
    2. Action
    3. PAUSE
    4. Observation
- Continue the Loop as long as the ai client returns Observation as response
- Stop the loop when ai client returns Answer instead of Observation

### Define System Prompt and Action mappings
- System Prompt for ReAct agent is to elaborate on how we want the ai client to behave
- Action mapping is a dictionary that let's ai client know about what actions/ api calls are available 

In [28]:
system_prompt= """ You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

average_score:
e.g. average score: 20 30 40 50
Invokes the function average_score  by passing these numbers as input parameter as a list of float type numbers. And returns the output of the function. These list of numbers represent individual heat score of a game.

heat_score:
e.g. heat score: 2 6 6 7 8
Invokes the function heat_score  by passing these numbers as input parameter as a list of float type numbers. And returns output of the function. These numbers represent various skill scores that an athelete receives by judges in an individual heat.

Example session:

Question: What is the average score of the athelete with the following heat scores: heat 1: 2 6 6 7 8 and heat 2 : 3 5 6 7 8
Thought: I should look the average score using heat_score
Action: heat_score: 2 6 6 7 8
PAUSE

You will be called again with this:

Observation: The heat score is 29 and 29 

You then output:

Answer: The heat score is 29 and 29 
""".strip()


In [33]:
# Define the functions to go into known actions
def average_score(scores):
    return scores

def heat_score(heat_scores):
    # Add the elements of the input list
    # total = 0
    # for i in heat_scores:
    #     total += float(i)
    return heat_scores

provisioned_actions = {
    "average_score": average_score,
    "heat_score": heat_score,
}


### Invoke the Agent class, that has mechanism to call AI

In [13]:
# Regular Expression to find Action 
action_reg_exp = re.compile(r'^Action: (\w+): (.*)$')

In [29]:
# Define the query function to invoke ai agent and invoke action if needed
def ai_query (question, max_turns=6):
    i = 0
    react_agent = Agent(system_prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        ai_response = react_agent(next_prompt)
        print(f"ai_response -> {ai_response}")
        # Find list of available actions
        actions = [
            action_reg_exp.match(a)
            for a in ai_response.split("\n")
            if action_reg_exp.match(a)
        ]
        print(f"actions: {actions}")

        if actions: # ai response indicates that action is required
            action, action_input = actions[0].groups()
            print(f"action: {action}")
            print(f"action_input: {action_input}")
            if action not in provisioned_actions:
                raise Exception(f"Action {action} with Action Group {action_input} not found")
            print(f"... Running {action} with {action_input}")
            observation = provisioned_actions[action](action_input)
            print(f"observation: {observation}")
            next_prompt = f"Observation: {observation}"
        else:
            return 
        


#### Invoke the Agent with a user prompt

In [34]:
user_prompt = """I have the following heat scores: heat One score is 2,6,6,7,8 and heat Two scpre is 3,5,6,7,8. 
What is the average score of the athelete with these heat scores?"""
ai_query(user_prompt)

ai_response -> Thought: I should calculate the average score using the heat scores provided for heat One and heat Two.

Action: heat_score: 2 6 6 7 8
PAUSE
actions: [<re.Match object; span=(0, 29), match='Action: heat_score: 2 6 6 7 8'>]
action: heat_score
action_input: 2 6 6 7 8
... Running heat_score with 2 6 6 7 8
observation: 2 6 6 7 8
ai_response -> Thought: Now, I need to calculate the average score for heat One.

Action: average_score: 2 6 6 7 8
PAUSE
actions: [<re.Match object; span=(0, 32), match='Action: average_score: 2 6 6 7 8'>]
action: average_score
action_input: 2 6 6 7 8
... Running average_score with 2 6 6 7 8
observation: 2 6 6 7 8
ai_response -> Answer: The average score for heat One is 5.8.
actions: []
